This first cell makes sure that torchgan is installed. It's primarily here for use with colab, since colab doesn't have torchgan installed by default.

In [1]:
# from torchGAN tutorial

try:
    import torchgan

    print(f"Existing TorchGAN {torchgan.__version__} installation found")
except ImportError:
    import subprocess
    import sys

    subprocess.check_call([sys.executable, "-m", "pip", "install", "torchgan"])
    import torchgan

    print(f"Installed TorchGAN {torchgan.__version__}")

Existing TorchGAN v0.1.0 installation found


In [1]:
import torch
import torch.nn as nn
import torch.backends.cudnn
import torch.utils.data as tdata
from torch.optim import Adam
import torchvision
import torchvision.datasets as dsets
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

import torchgan
from torchgan.models import *
from torchgan.losses import *
from torchgan.trainer import Trainer

This cell takes care of creating the dataset. Any PNG or JPEG image files are acceptable. The models require that the width and height of the images are equal, and that the dimensions of the image are a perfect power of 2, but those can both be overcome by scaling the images.

The size variable is how big the generated images will be. Larger images can be more detailed, but dramatically slow down the training process. Additionally, as mentioned above, this value must be a perfect power of 2.

If your images are in black and white, or if you wish to convert them to black and white, set grayscale to True in line 2.

Due to the way that torchvision works, the file path provided in line 3 cannot be the exact folder that holds the images. In order for the path to be recognized properly, you must specify the folder that contains the folder with the images.

For copyright reasons, the images that I am training from are not included with the GitHub repo. If you are interested in learning more, the citations for all of the images are included in the bibliography of the accompanying report.

In [2]:
size = 128
grayscale = False   # change this to True if your images are in black and white
folder = "./data/otherGANGogh/images/landscape/"

if grayscale:
    channels = 1
    data = ImageFolder(
        root=folder,
        transform=transforms.Compose([
            transforms.Resize((size, size)),
            transforms.ToTensor(),
            transforms.Grayscale(),
            transforms.Normalize(mean=.5, std=.5)
        ])
    )
else:
    channels = 3
    data = ImageFolder(
        root=folder,
        transform=transforms.Compose([
            transforms.Resize((size, size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=.5, std=.5)
        ])
    )
print(len(data))

15000


In [3]:
dataloader = tdata.DataLoader(data, batch_size=64, shuffle=True)

In [4]:
# adapted from torchgan tutorial

dcgan_network = {
    "generator": {
        "name": DCGANGenerator,
        "args": {
            "encoding_dims": 100,
            "out_size": size,           # this is how many pixels wide and tall the generated images will be
            "out_channels": channels,
            "step_channels": 32,
            "nonlinearity": nn.LeakyReLU(0.2),
            "last_nonlinearity": nn.Tanh(),
        },
        "optimizer": {"name": Adam, "args": {"lr": 0.0001, "betas": (0.5, 0.999)}},
    },
    "discriminator": {
        "name": DCGANDiscriminator,
        "args": {
            "in_size": size,            # this is how many pixels wide and tall we resized the images to be.
            "in_channels": channels,
            "step_channels": 32,
            "nonlinearity": nn.LeakyReLU(0.2),
            "last_nonlinearity": nn.LeakyReLU(0.2),
        },
        "optimizer": {"name": Adam, "args": {"lr": 0.0003, "betas": (0.5, 0.999)}},
    },
}

In [5]:
# from torchgan tutorial

minimax_losses = [MinimaxGeneratorLoss(), MinimaxDiscriminatorLoss()]
wgangp_losses = [
    WassersteinGeneratorLoss(),
    WassersteinDiscriminatorLoss(),
    WassersteinGradientPenalty(),
]
lsgan_losses = [LeastSquaresGeneratorLoss(), LeastSquaresDiscriminatorLoss()]

If you're running this on colab, be sure to select a GPU runtime for the best performance.

In [6]:
# more from torchgan tutorial

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    # Use deterministic cudnn algorithms
    torch.backends.cudnn.deterministic = True
    epochs = 10
else:
    device = torch.device("cpu")
    epochs = 5

print("Device: {}".format(device))
print("Epochs: {}".format(epochs))

Device: cpu
Epochs: 5


In [17]:
# from torchgan tutorial
epochs = 700    # you can override the number of epochs here if you want to try to get better results

trainer = Trainer(
    dcgan_network,
    lsgan_losses,
    sample_size=64,
    epochs=epochs,
    device=device
)

This is the cell that actually does the training. After it finishes, use the next cell if you want to keep training the same model.

In [8]:
trainer(dataloader)

c:\users\leafy\pycharmprojects\gangogh\venv\lib\site-packages\PIL\TiffImagePlugin.py:845: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


Saving Model at './model/gan0.model'
Epoch 1 Summary
Epoch time duration : 1003.3898904323578
generator Mean Gradients : 36.18075446934572
discriminator Mean Gradients : 367.2088514130637
Mean Running Discriminator Loss : 0.36510538287619326
Mean Running Generator Loss : 0.8458863792901343


c:\users\leafy\pycharmprojects\gangogh\venv\lib\site-packages\torchvision\utils.py:68: UserWarning: The parameter 'range' is deprecated since 0.12 and will be removed in 0.14. Please use 'value_range' instead.
  warnings.warn(


Generating and Saving Images to ./images/epoch1_generator.png

Saving Model at './model/gan1.model'
Epoch 2 Summary
Epoch time duration : 856.2223539352417
generator Mean Gradients : 19.761261659166436
discriminator Mean Gradients : 256.79356252814733
Mean Running Discriminator Loss : 0.2177339577472749
Mean Running Generator Loss : 0.7154120697937113
Generating and Saving Images to ./images/epoch2_generator.png

Saving Model at './model/gan2.model'
Epoch 3 Summary
Epoch time duration : 669.5286500453949
generator Mean Gradients : 13.850664039708674
discriminator Mean Gradients : 220.01047040482334
Mean Running Discriminator Loss : 0.1688748215080787
Mean Running Generator Loss : 0.6690223753663666
Generating and Saving Images to ./images/epoch3_generator.png

Saving Model at './model/gan3.model'
Epoch 4 Summary
Epoch time duration : 720.5223140716553
generator Mean Gradients : 10.758290492662736
discriminator Mean Gradients : 191.93496075266353
Mean Running Discriminator Loss : 0.1394

Use this cell instead of the previous one if you want to resume training a model that you saved from a previous session. Be sure to use the correct file path and file name when you call the function.

In [18]:
model = "./model/gan4.model"

trainer.load_model(model)
trainer(dataloader)

Loading Model From './model/gan4.model' to cpu
Saving Model at './model/gan0.model'
Epoch 601 Summary
Epoch time duration : 896.8301000595093
generator Mean Gradients : 358.1461636051608
discriminator Mean Gradients : 0.577842693512792
Mean Running Discriminator Loss : 0.017121337555572182
Mean Running Generator Loss : 0.5313584453214201
Generating and Saving Images to ./images/epoch1_generator.png

Saving Model at './model/gan1.model'
Epoch 602 Summary
Epoch time duration : 796.395012140274
generator Mean Gradients : 336.4234823853853
discriminator Mean Gradients : 0.5655728990364813
Mean Running Discriminator Loss : 0.01710568771397569
Mean Running Generator Loss : 0.5313479043554399
Generating and Saving Images to ./images/epoch2_generator.png

Saving Model at './model/gan2.model'
Epoch 603 Summary
Epoch time duration : 639.3975124359131
generator Mean Gradients : 354.99015876436545
discriminator Mean Gradients : 0.60791500271883
Mean Running Discriminator Loss : 0.01709160197921850